In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "QQQ"
inverse = "PSQ"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [2]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2021-12-27,10.786800,PSQ,long,enter,0.059081,4938.158399
1,2022-03-07,13.150200,PSQ,long,exit,0.944378,6064.202390
2,2022-03-07,328.108585,QQQ,long,enter,0.944378,5990.567086
3,2022-03-28,368.559104,QQQ,long,exit,0.039121,6760.566057
4,2022-03-28,11.604900,PSQ,long,enter,0.039121,6676.907743
5,2022-05-09,14.018800,PSQ,long,exit,0.908016,8125.778436
6,2022-05-09,300.121494,QQQ,long,enter,0.908016,8025.233492
7,2022-06-02,317.523805,QQQ,long,exit,0.291390,8553.893533
8,2022-06-02,13.119900,PSQ,long,enter,0.291390,8448.073800
9,2022-06-13,14.907600,PSQ,long,exit,0.980558,9670.289249


In [3]:
df2.tail(365)


,Open,High,Low,Close,Adj Close,Volume,prob_above_trend,weighted-volume,trend,long_entry,long_exit,short_entry,short_exit
Date,,,,,,,,,,,,,
2021-12-16,398.040009,398.480011,385.209991,386.839996,382.927307,82591400,0.361672,3.194966e+10,380.138667,NaN,NaN,NaN,NaN
2021-12-17,383.329987,389.190002,381.769989,384.910004,381.016846,80492500,0.418821,3.098237e+10,379.810167,NaN,NaN,NaN,NaN
2021-12-20,379.570007,381.500000,377.470001,380.690002,377.320862,65336800,0.508362,2.487307e+10,379.445389,NaN,NaN,NaN,NaN
2021-12-21,384.049988,389.529999,380.200012,389.209991,385.765411,48899500,0.333859,1.903217e+10,379.138000,NaN,NaN,NaN,NaN
2021-12-22,388.910004,394.160004,388.230011,393.950012,390.463501,37042200,0.217624,1.459278e+10,378.829389,NaN,NaN,NaN,NaN
2021-12-23,394.309998,398.429993,394.140015,396.920013,393.407227,29595500,0.162504,1.174705e+10,378.484055,NaN,NaN,NaN,NaN
2021-12-27,398.250000,403.480011,398.220001,403.480011,399.909149,32820700,0.059081,1.324250e+10,378.131055,NaN,NaN,403.480011,NaN
2021-12-28,404.399994,404.579987,400.709991,401.609985,398.055695,33113700,0.077102,1.329879e+10,377.787222,NaN,NaN,NaN,NaN
2021-12-29,401.750000,403.049988,399.109985,401.549988,397.996216,32248500,0.064661,1.294938e+10,377.384611,NaN,NaN,NaN,NaN


In [5]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
env.expert_opinion_df()
env.timeseries.tail(35)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Close,weighted-volume,trend,prob_above_trend,action
Date,,,,,
2023-04-12,0.369229,0.231429,0.323330,0.390292,0
2023-04-13,0.411982,0.197482,0.326837,0.297142,0
2023-04-14,0.407797,0.205048,0.330343,0.484024,0
2023-04-17,0.409680,0.140922,0.333849,0.548876,0
2023-04-18,0.409820,0.159072,0.337355,0.550923,0
2023-04-19,0.408774,0.121220,0.340861,0.629635,0
2023-04-20,0.391826,0.232835,0.344367,0.689404,0
2023-04-21,0.394127,0.216488,0.347873,0.812121,0
2023-04-24,0.389524,0.144010,0.351379,0.864495,0
